## A comparison of Pythereum to Web3.py in performance, functionality and ease of use

In [1]:
import asyncio

import pythereum as pye

from eth_account import Account
from time import time, sleep
from web3 import AsyncWeb3, AsyncHTTPProvider, WebsocketProviderV2
from ens import AsyncENS
from dotenv import dotenv_values
from pprint import pprint

In [2]:
# Test parameters
iterations = 1_000_000

connection_url = "127.0.0.1:8545"
external_connection_url = dotenv_values("../.env")["TEST_WS"]
http_external_url = external_connection_url.replace("wss://", "https://")

w3 = AsyncWeb3(AsyncHTTPProvider(http_external_url))
w3_local = AsyncWeb3(AsyncHTTPProvider(f"http://{connection_url}"))

In [4]:
nsum = 0
pyetimesum = 0
w3timesum = 0


for n in range(100, 5100, 1000):
    t0 = time()

    async with pye.EthRPC(f"ws://{connection_url}") as erpc:
        await erpc.get_block_by_number(
            list(range(n)),
            [True for i in range(n)]
        )
    print(f"pythereum processed {n} get_block_by_number requests in {(pyetime := time() - t0)} seconds")
    print(f"that's {n / pyetime} requests per second!")

    t0 = time()

    for result in asyncio.as_completed(
            [w3_local.eth.get_block(num, True) for num in range(n)]
        ):
        try:
            await result
        except:
            pass

    print(f"Web3.py processed {n} get_block_by_number requests in {(w3time := time() - t0)} seconds")
    print(f"that's {n / w3time} requests per second")
    
    nsum += n
    pyetimesum += pyetime
    w3timesum += w3time
    
print(f"\nOn average pythereum processed {nsum / pyetimesum} requests per second and web3.py processed {nsum / w3timesum} requests per second\nThat's a {w3timesum/pyetimesum} times speedup!")
    

pythereum processed 100 get_block_by_number requests in 0.02303791046142578 seconds
that's 4340.671440990189 requests per second!
Web3.py processed 100 get_block_by_number requests in 0.04408693313598633 seconds
that's 2268.2457791189418 requests per second
pythereum processed 1100 get_block_by_number requests in 0.19179439544677734 seconds
that's 5735.308362048819 requests per second!
Web3.py processed 1100 get_block_by_number requests in 0.40035295486450195 seconds
that's 2747.5755745884057 requests per second
pythereum processed 2100 get_block_by_number requests in 0.26764893531799316 seconds
that's 7846.098836541211 requests per second!
Web3.py processed 2100 get_block_by_number requests in 0.7279794216156006 seconds
that's 2884.696926376685 requests per second
pythereum processed 3100 get_block_by_number requests in 0.47873473167419434 seconds
that's 6475.402336402287 requests per second!
Web3.py processed 3100 get_block_by_number requests in 1.1260836124420166 seconds
that's 2752

### Individual call speeds (Not making use of batching)

In [11]:
t0 = time()

num_calls = 100

async with pye.EthRPC(external_connection_url, use_socket_pool=False) as erpc:
    async with asyncio.TaskGroup() as tg:
        for i in range(num_calls):
            tg.create_task(erpc.get_block_number())
            
print((pyetime_individual := time() - t0))
t0 = time()

# Since web3.py is single-socketed it will raise too many requests errors on many free endpoints with higher than 80 task pairs
async with asyncio.TaskGroup() as tg:
    for i in range(num_calls):
        tg.create_task(w3.eth.get_block_number())
            
print((w3time_individual := time() - t0))
t0 = time()

async with AsyncWeb3.persistent_websocket(WebsocketProviderV2(external_connection_url)) as ws:
    async with asyncio.TaskGroup() as tg:
        for i in range(num_calls):
            tg.create_task(ws.eth.get_block_number())
    
print((w3time_websocket_individual := time() - t0))

print(f"When calling {num_calls} functions in the same way using task groups, Pythereum is found to be {w3time_individual / pyetime_individual} times faster than Web3.py")
print(f"Additionally it is {w3time_websocket_individual / pyetime_individual} times faster than a websocketed version of Web3.py")
print("From running this many times Pythereum averages a 2.5-4x speedup over Web3.py")

1.4234800338745117
2.466068744659424
2.844428539276123
When calling 100 functions in the same way using task groups, Pythereum is found to be 1.7324224337359568 times faster than Web3.py
Additionally it is 1.998221591864545 times faster than a websocketed version of Web3.py
From running this many times Pythereum averages a 2.5-4x speedup over Web3.py


### More intensive tests, with higher iteration numbers, both processed using asyncio.gather

In [4]:
t0 = time()

# Pythereum using batch calls

async with pye.EthRPC(f"ws://{connection_url}", pool_size=10) as erpc:
    await asyncio.gather(*(
        erpc.get_block_by_number(
            [i for i in range(start, start+1000)], 
            [True for i in range(start, start+1000)]
        ) for start in range(0, iterations, 1000)
    ))
print(f"Pythereum made {iterations} requests in {(pyetime_intense := time() - t0)} seconds")

Pythereum made 1000000 requests in 134.84792470932007 seconds


In [5]:
t0 = time()
# Web3 does not natively support batch RPC calls

async def blknum(num):
    # Defined to handle errors as Web3.py does not like empty returns
    # Yes it is better to have an exception type, no it doesnt really matter
    try:
        await w3_local.eth.get_block(num, True)
    except:
        pass

await asyncio.gather(*(blknum(num) for num in range(iterations)))
print(f"Web3.py made {iterations} requests in {(w3time_intense := time() - t0)} seconds")

Web3.py made 1000000 requests in 377.48311281204224 seconds


In [6]:
print(f"Pythereum is able to process transactions {w3time_intense / pyetime_intense} times faster than Web3.py in intense testing,\nWith Pythereum processing {iterations / pyetime_intense} transactions per second,\nand Web3 processing {iterations / w3time_intense} transactions per second")

Pythereum is able to process transactions 2.79932459936443 times faster than Web3.py in intense testing,
With Pythereum processing 7415.7611409713045 transactions per second,
and Web3 processing 2649.1251292025972 transactions per second


### Subscriptions

In [11]:
# Pythereum subscription implementation, an infinite loop which gets block headers whenever new blocks are created

# Web3.py has no native implementation of subscriptions, and as such would have to be coded from scratch

# Items will be in the form of a Block object, with each json value converted into an appropriate data type

# Running this on localhost will result in nothing happening as no blocks are being created!

async with pye.EthRPC(external_connection_url) as erpc:
    async with erpc.subscribe(pye.SubscriptionType.new_heads) as sc:
        async for item in sc.recv():
            pprint(item)

Block(difficulty=0,
      extra_data=HexStr('0x7273796e632d6275696c6465722e78797a'),
      gas_limit=30000000,
      gas_used=12199687,
      hash=HexStr('0x537ee530093d60fabe3120d2222f839653d0cfde31ea0d94686be44feade76de'),
      logs_bloom=HexStr('0x90271110f7b1a37ad05a0785d1e259b7b3d1498bcbb50c89828920087732b64a00c071e865558062c6381f63623e51100a03e0268a04bc244e42052cc37e7f0529415b6a4d308a397812532ff484696184c99e85186f3c45c000b420bcbfbec00b0c8c7e1a8bd302af4ce23109313f6526000e290c689728f6d656d60809542906f427db4424b87f808a51730d3a46ee0235521dcd05c52c7579007f00b974308b2a437c1583edcbcbbd69e4988e9598262c065133a6ab0f0d66261790ebd123cb192c3294a816ca4389d5d2c288969e0cde30de4ee901f04f37880e8084e048d0f7e60c8bd082681ba49aa1884848d45d29d3d23cec1044c34db818664f54d8'),
      miner=HexStr('0x1f9090aae28b8a3dceadf281b0f12828e676c326'),
      mix_hash=HexStr('0x20bfcb2fd0b52112a27939675cd5e74c17ca0afd2891dc91a2e383c93fc21413'),
      nonce=0,
      number=18415340,
      parent_hash=HexStr('0x02feed3

Block(difficulty=0,
      extra_data=HexStr('0x47616d626974204c616273202868747470733a2f2f676d6269742e636f29'),
      gas_limit=30000000,
      gas_used=17301091,
      hash=HexStr('0x71d2e24813f5a6fbfd2f97220750a33a29eb77fe7084229a4ee0eec65ca3c774'),
      logs_bloom=HexStr('0xe733951561a27169d65a9986c9fef331435bc9a16ca96405858bd660f7b159922472599b045179b7f197bf36617f91f46e09ac099effade676fb9d09d92a3c036c736ccced6cbee8e80bd13fdda82be897b5466bcd6a1a275f1d1d409b353bf79e612446928eda242cd0d188200f8edba75cc6f3b038d700be964c9e496e63588ff733df22b54bb38dffe46d453ae3ce33751ed7ed2be16c6431504b339fcd60ffe4fd7eb9c6e173bb34f2c59895b5ca5e48469178ae894b8fe7f7ea96779261de0b34a6d07e204853d382d04ffffed4a6e66a87de62135ded39b80e18b5f458f779ea493910446acecda79538d3c76417e4d1f8ebbe11f2fb0f00d0514b76b1'),
      miner=HexStr('0x388c818ca8b9251b393131c08a736a67ccb19297'),
      mix_hash=HexStr('0x2fa0e340ab2f22a3a1dfba31ea1e707054c289dbca44d40485f07de38bda6c60'),
      nonce=0,
      number=18415346,
      par

CancelledError: 

### 